In [1]:
# ! git clone https://github.com/naftalima/androidGames.git

In [2]:
# cd androidGames/

## O .csv da Query
* está em ./data/
    >_SELECT_name_battery_level_version_name_device_id_timestamp_FROM_201909111900.csv

In [12]:
import pandas as pd
import numpy as np
import math

#df: data frame
df = pd.read_csv('data/_SELECT_name_battery_level_version_name_device_id_timestamp_FROM_201909111900.csv')
# name, battery_level, version_name, device_id, timestamp

# str into  <class 'pandas._libs.tslib.Timestamp>
df['timestamp'] = pd.to_datetime(df['timestamp'])

df.head(10)

,name,battery_level,version_name,device_id,timestamp
0,com.dts.freefireth,0.89,1.17.1,3383,2018-06-17 03:55:49
1,com.dts.freefireth,0.80,1.17.1,3383,2018-06-17 04:13:19
2,com.dts.freefireth,0.80,1.17.1,3383,2018-06-17 04:13:49
3,com.dts.freefireth,0.79,1.17.1,3383,2018-06-17 04:14:19
4,com.dts.freefireth,0.79,1.17.1,3383,2018-06-17 04:14:49
5,com.dts.freefireth,0.79,1.17.1,3383,2018-06-17 04:15:19
6,com.dts.freefireth,0.78,1.17.1,3383,2018-06-17 04:15:49
7,com.dts.freefireth,0.78,1.17.1,3383,2018-06-17 04:16:19
8,com.dts.freefireth,0.44,1.17.1,3383,2018-06-17 05:17:47
9,com.dts.freefireth,0.44,1.17.1,3383,2018-06-17 05:18:17


In [4]:
def games_names():
    f = open("data/jogos.txt", "r")
    jogos=[]
    for line in f:
        linha = line.split(": ")
        jogos.append([linha[0],linha[1]])
    f.close()
    for  i in range(len(jogos)):
        jogos[i][1] = jogos[i][1].replace('\n','')
    return(jogos)

def url(jogosnomes):
    urls= []
    for i in jogosnomes:
        urls.append(i[1])
    return(urls)

def GamesMostUsed(listOfGames):
    """Os com mais dados do top 20 da playstore"""
    amount = []
    for i in listOfGames:
        qtd=0
        for index, row in df.iterrows():
            if row["name"] == i:
                qtd+=1
        amount.append(qtd)
        # print(i,"qtd: ",qtd)
    gamesGood = []
    for i in range(len(amount)):
        if amount[i] > 1000:
            gamesGood.append(listOfGames[i])
    return(gamesGood)


## Os jogos com maior quantidade de dados:

In [5]:
games = GamesMostUsed(url(games_names()))
print(games)

['com.dts.freefireth', 'com.supercell.clashroyale', 'com.nianticlabs.pokemongo', 'com.supercell.clashofclans', 'com.king.candycrushsaga', 'com.mobile.legends', 'com.tencent.ig', 'com.miniclip.eightballpool', 'com.playrix.township', 'com.king.candycrushsodasaga', 'com.kiloo.subwaysurf', 'com.imangi.templerun2']


## Agrupando os dados de cada jogo:
* Cada jogo terá um dicionario, onde a chave é a versão do jogo. Cada item é outro dicionario, em que a chave é o device_id, i.e. um dispositivo diferente.
* O item de destes dicionarios é uma lista com dados:
    > [ timestamp , battery_level   ]
## Por uso:
* o dado é adicionada a cada decaimento de 1% na bateria, então uma margem de 3% de decaimento.
<p> salvando em duas listas, battery level e timestamp inicial e final respectivamente, para cada uso.
    salvando no dataframe o qnto foi utilizado, diferença, em cada uso.
</p>

In [6]:
def agrupando(nome):
    # version_name={ #NESTED DICT
    #     'Version_Name': {'Device_Id': [ [timestamp, battery level] ] }
    #                                        #NESTED LIST
    version_name={0.0: {0: []}}
    for index, row in df.iterrows():
        if row["name"] == nome:  #para todos os dados do jogo selecionado
            #agrupando pela versão
            if not row["version_name"] in version_name:
                version_name[row["version_name"]] = {} # cria
            #agrupando pelo dispositivo
            if not row['device_id'] in version_name[row["version_name"]]:
                version_name[row["version_name"]][row['device_id']] = [] #cria
            version_name[row["version_name"]][row['device_id']].append([row['timestamp'],row['battery_level']]) #insere
    del version_name[0.0]
    return(version_name)

def porUso(jogoAgrupado):
    """Separando por uso e deletando os repetidos"""
    dfJogo = pd.DataFrame()
    dateInit = []
    batteryInit =[]
    dateFinal = []
    batteryFinal =[]
    for key, val in jogoAgrupado.items():
        # print("version_name",key)
        for key0, val0 in val.items():
            # print("device_id", key0)
            n = len(val0)
            for i in range(n-1):
                if  i == 0: #inicio da lista
                    dateInit.append(val0[i][0])
                    batteryInit.append(val0[i][1])
                    # print("START","inicio: ", val0[i][1])
                diff =  val0[i+1][1] - val0[i][1]
                if ( diff < -0.03 ) or ( diff > 0.0): #parou
                    dateFinal.append(val0[i][0])
                    batteryFinal.append(val0[i][1])
                    dateInit.append(val0[i+1][0])
                    batteryInit.append(val0[i+1][1])
                    # print("STOP","fim: ", val0[i][0], "inicio do prox:", val0[i+1][0])
                if i == n-2: ##fim da lista
                    dateFinal.append(val0[i+1][0])
                    batteryFinal.append(val0[i+1][1])
                    # print("END", "final:", val0[i+1][1])
    # print("inicio: ", batteryInit, dateInit)
    # print("fim: ",batteryFinal,dateFinal)
    dfJogo['BatteryLevelFinal'] = batteryFinal
    dfJogo['BatteryLevelInitial'] = batteryInit
    dfJogo['TimestampFinal'] = dateFinal
    dfJogo['TimestampInitial'] = dateInit
    # print(dfJogo.head(10))
    batteryused = []
    elapsedtime = []
    for index, row in dfJogo.iterrows():
        battery = np.subtract(row['BatteryLevelInitial'], row['BatteryLevelFinal'] )
        time = np.subtract( row['TimestampFinal'] , row['TimestampInitial'] )
        # # <class 'pandas._libs.tslib.Timedelta'>
        # # diff.seconds <class 'int'>
        batteryused.append(battery)
        elapsedtime.append(time.seconds)
    dfJogo = dfJogo.drop(columns="BatteryLevelFinal")
    dfJogo = dfJogo.drop(columns="BatteryLevelInitial")
    dfJogo = dfJogo.drop(columns="TimestampInitial")
    dfJogo = dfJogo.drop(columns="TimestampFinal")

    dfJogo['Battery_Used'] = batteryused
    dfJogo['ElapsedTimestamp'] = elapsedtime
    dfJogo = dfJogo.set_index("Battery_Used")
    dfJogo = dfJogo.drop(0, axis=0)

    return(dfJogo)

### Neste formato:
* ElapsedTimestamp: segundos
* Battery_Ussed:
    100% -> 1.00
    
### Arquivo:
* local:
    > ./data/


In [14]:
for i in games:
    dfG = porUso(agrupando(i))
    print(i)
    print( dfG.head(5))

com.dts.freefireth
              ElapsedTimestamp
Battery_Used                  
0.01                        60
0.02                       193
0.01                       348
0.15                       511
0.01                        24
com.supercell.clashroyale
              ElapsedTimestamp
Battery_Used                  
0.06                      1024
0.01                       107
0.04                       447
0.12                      1775
0.17                      3362
com.nianticlabs.pokemongo
              ElapsedTimestamp
Battery_Used                  
0.01                       197
0.01                       172
0.01                       250
0.02                       451
0.02                       679
com.supercell.clashofclans
              ElapsedTimestamp
Battery_Used                  
0.13                     35806
0.02                        90
0.16                      1295
0.08                       632
0.12                       451
com.king.candycrushsaga
          

In [21]:
def Count_Users_Versions():
    users_versions = {}
    for i in games:
        nameG = i
        dictG = agrupando(nameG)
        versoes = len(dictG)
        users = 0
        for key in dictG: #key == versionname
            users += len(dictG[key])
        users_versions[nameG] = [versoes,users]
    df_uv = pd.DataFrame.from_dict(users_versions,orient='index')
    df_uv.columns = ['versoes','users']
    return(df_uv)

### Quantidade de Usuarios e Versões para cada jogo analisado:

In [23]:
dfCounter =  Count_Users_Versions()
dfCounter.head(12)

,versoes,users
com.nianticlabs.pokemongo,18,22
com.supercell.clashofclans,36,144
com.king.candycrushsaga,31,54
com.king.candycrushsodasaga,12,17
com.kiloo.subwaysurf,33,141
com.supercell.clashroyale,20,73
com.imangi.templerun2,23,43
com.miniclip.eightballpool,15,45
com.dts.freefireth,24,320
com.tencent.ig,8,37


#### Com mais:

In [24]:
dfCounter.idxmax()

versoes    com.mobile.legends
users      com.dts.freefireth
dtype: object

#### Com menos:

In [26]:
dfCounter.idxmin()

versoes    com.playrix.township
users      com.playrix.township
dtype: object

# Normalizando

In [33]:
def por_minuto():
    """Normalizando os dados por uso em qnto % de bateria é utilizado por minuto"""
    dado={'nome':[]}
    for i in range(len(games)):
        csv = 'data/' + games[i] + '.csv'
        df = pd.read_csv(csv)
        batteryPerMinute = []
        for index, row in df.iterrows():
            minuto = np.divide(row['ElapsedTimestamp'] , 60 )
            batmin =  np.divide(row['Battery_Used'], minuto)
            if batmin < 0.05:# isn't outlier
                batteryPerMinute.append(batmin)
        dado[games[i]] = batteryPerMinute
    del dado['nome']
    return(dado)

def Medicoes(dado):
    metricas = {} #{'nome_jogo': media, mediana, desvio_padrao}
    for i in games:
        metricas[i] = [np.mean(dado[i]),np.median(dado[i]),np.std(dado[i])]
    metricas = pd.DataFrame.from_dict(metricas,orient='index')
    metricas.columns = ['media','mediana','std']
    return(metricas)

In [34]:
data = por_minuto()
medidas = Medicoes(data)
medidas.head(12)

,media,mediana,std
com.nianticlabs.pokemongo,0.006743,0.006507,0.004452
com.supercell.clashofclans,0.007801,0.006667,0.006269
com.king.candycrushsaga,0.008836,0.005125,0.009271
com.king.candycrushsodasaga,0.017145,0.015409,0.013084
com.kiloo.subwaysurf,0.007806,0.006667,0.006344
com.supercell.clashroyale,0.006154,0.004800,0.005569
com.imangi.templerun2,0.012701,0.008571,0.009605
com.miniclip.eightballpool,0.008768,0.007500,0.006156
com.dts.freefireth,0.009903,0.007429,0.008592
com.tencent.ig,0.013221,0.011502,0.010479


In [36]:
medidas.idxmax()

media      com.king.candycrushsodasaga
mediana    com.king.candycrushsodasaga
std        com.king.candycrushsodasaga
dtype: object

In [37]:
medidas.idxmin()

media      com.supercell.clashroyale
mediana    com.supercell.clashroyale
std        com.nianticlabs.pokemongo
dtype: object

# Por versão